In [1]:
# Import useful packages
import os
import time
import pandas as pd
# Suppressing some warnings in pandas
pd.options.mode.chained_assignment = None
import numpy as np
import glob
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime, date, time, timedelta
from ta import add_all_ta_features

# Importing our self-created functions
from feature_creation import *
from portfolio import *

# For loading/querying postgres
from sqlalchemy import create_engine
import psycopg2
from psycopg2 import connect

In [2]:
# # **Only used when re-creating feature_df from scratch (time consuming)**
# # Creating stock_df from feature file
# stock_df = daily_features()
# recommendations_df = pd.read_csv('assets/ticker_recommendations.csv')
# stock_df.head()

In [3]:
# # **Only used when re-creating feature_df from scratch (time consuming)**
# # Creating feature_df from scratch, but computing technical analysis features for all tickers

# # Create list of tickers for creating technical features and later usage
# tickers = list(stock_df.ticker.unique())

# # Running technical analysis feature creation for all tickers individually

# # count = 0

# for ticker in tickers:
    
#     temp_df = stock_df[stock_df['ticker']==ticker]
    
#     temp_df = add_all_ta_features(
#     temp_df, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)
    
#     if count == 0:
#         feature_df = temp_df
#         count += 1
        
#     else:
#         feature_df = pd.concat([feature_df,temp_df])
#         count += 1
#     print(f'Finished {ticker}. Done with {count} of 504 tickers')
    
# # save feature df to csv for future usage. Feature creation takes hours on entire dataset
# feature_df.to_csv('assets/models/tyler_rf_daily_update/ta_feature_df.csv')

In [4]:
# Or just load if no need to re-create features
feature_df = pd.read_csv('assets/models/tyler_rf_daily_update/ta_feature_df.csv',index_col=0)

# Ensure index is datetime
feature_df.index = pd.to_datetime(feature_df.index)

feature_df.head()

,Adj Close,Close,Close_adj,Dividends,High,High_adj,Low,Low_adj,Open,Open_adj,...,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr
Date,,,,,,,,,,,,,,,,,,,,,
1990-02-16,0.056967,0.056967,0.077257,0.0,0.058887,0.079861,0.054407,0.073785,0.0,0.0,...,-42.857143,0.0,0.056967,0.0,0.000000,0.000000,0.000000,-99.670356,0.000000,0.000000
1990-02-20,0.058887,0.058887,0.079861,0.0,0.058887,0.079861,0.055047,0.074653,0.0,0.0,...,-0.000804,0.0,0.057802,0.0,-7.132343,-1.426469,-5.705874,3.370505,3.314949,3.370505
1990-02-21,0.057607,0.057607,0.078125,0.0,0.058247,0.078993,0.055687,0.075521,0.0,0.0,...,-28.571863,0.0,0.057718,0.0,-14.352665,-4.011708,-10.340957,-2.173750,-2.197724,1.123489
1990-02-22,0.058247,0.058247,0.078993,0.0,0.060167,0.081597,0.058247,0.078993,0.0,0.0,...,-33.333463,0.0,0.057944,0.0,-21.164525,-7.442271,-13.722254,1.111033,1.104906,2.247004
1990-02-23,0.057927,0.057927,0.078559,0.0,0.058887,0.079861,0.057607,0.078125,0.0,0.0,...,-38.889100,0.0,0.057937,0.0,-27.325018,-11.418821,-15.906197,-0.549422,-0.550937,1.685236


In [5]:
# Create SQL engine to load data to postgres for central usage
from sqlalchemy import create_engine
dbname = '697_temp'
user = 'postgres'
password = 'poRter!5067'
host = 'databasesec.cvhiyxfodl3e.us-east-2.rds.amazonaws.com'

engine = create_engine(f'postgresql://{user}:{password}@{host}/{dbname}')

In [10]:
# create temp_df to test loading
temp_df = feature_df[(feature_df.index >= '2017-01-01') & (feature_df.index <= '2021-07-01')]

temp_df.to_sql('technical_features_daily', engine)